In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Load dataset

In [496]:
df = pd.read_csv('data/hist_ppgi.csv', sep=';',encoding='latin-1')

## Remover as disciplinas que não são relevantes no PPGI

In [497]:
flag_ppgi = copia['Código da Disciplina'] >= 316000

flag_ppgi2 = copia['Código da Disciplina'] < 317000

df = df[flag_ppgi & flag_ppgi2]

### Remove coluna do nome do aluno

In [498]:
matricula = df['Matrícula']

periodo = df['Período']

matdisano = periodo // 10

matdissem = periodo % 10

semestre = (matricula // 100000)%10+1

ingresso = 2000+(matricula // 10000000)

#df['Ingresso'] = ingresso*10+semestre

df['Tempo'] = (matdisano - ingresso)*2 + matdissem-semestre+1

In [499]:
#df = df.drop(['Nome','Curso','Código do Curso','Modalidade','Turma','Código da Opção','Nível'], axis=1)

nroInstances,nroAttributes = df.shape

df

,Período,Matrícula,Nome,Código da Disciplina,Disciplina,Turma,Menção,Modalidade,Curso,Código do Curso,Código da Opção,Nível,Tempo
0,20102,100000541,Alessandro Ferreira Leite ...,316261,Tópicos em Engenharia de Software ...,A,SS,OPT,Informática ...,51799,2631,Doutorado,2
1,20102,100000541,Alessandro Ferreira Leite ...,316440,Sistemas Distribuídos ...,A,MS,OPT,Informática ...,51799,2631,Doutorado,2
2,20102,100132855,Shana Schlottfeldt Santos ...,316032,Inteligência Artificial 2 ...,A,SS,OPT,Informática ...,51799,2631,Doutorado,1
3,20102,100132855,Shana Schlottfeldt Santos ...,316504,Tópicos em Fundamentos e Métodos de Computação...,A,MS,OPT,Informática ...,51799,2631,Doutorado,1
4,20102,100132871,Wosley da Costa Arruda ...,316296,Teoria da Computação ...,A,MM,OBR,Informática ...,51799,2631,Doutorado,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,20192,190144661,Sérgio Augusto Pará Bittencourt Neto ...,316377,Fundamentos de Sistemas Computacionais ...,A,MM,OBS,Informática ...,51799,2631,Doutorado,1
1141,20192,190144688,Walter Lucas Monteiro de Mendonça ...,316261,Tópicos em Engenharia de Software ...,A,MS,OPT,Informática ...,51799,2631,Doutorado,1
1142,20192,190144688,Walter Lucas Monteiro de Mendonça ...,316415,Seminário ...,A,SS,OBR,Informática ...,51799,2631,Doutorado,1
1143,20192,190147661,Jessé Barreto de Barros ...,316296,Teoria da Computação ...,M,SR,OBS,Informática ...,51799,2631,Doutorado,1


## Erros de preenchimento

In [500]:
flag_tempo = df['Tempo'] < 0

erros = df[flag_tempo]

erros

,Período,Matrícula,Nome,Código da Disciplina,Disciplina,Turma,Menção,Modalidade,Curso,Código do Curso,Código da Opção,Nível,Tempo
11,20102,120048582,Edans Flávius de Oliveira Sandes ...,316415,Seminário ...,A,SS,OPT,Informática ...,51799,2631,Doutorado,-2
12,20102,120048582,Edans Flávius de Oliveira Sandes ...,316504,Tópicos em Fundamentos e Métodos de Computação...,A,MS,OPT,Informática ...,51799,2631,Doutorado,-2
13,20102,120048582,Edans Flávius de Oliveira Sandes ...,316547,Proficiência em Inglês ...,A,AP,OBR,Informática ...,51799,2631,Doutorado,-2
14,20102,120099551,Zheng Jianya ...,316032,Inteligência Artificial 2 ...,A,MM,OPT,Informática ...,51799,2631,Doutorado,-2
15,20102,120099551,Zheng Jianya ...,316296,Teoria da Computação ...,A,MS,OBR,Informática ...,51799,2631,Doutorado,-2
16,20102,120099551,Zheng Jianya ...,316440,Sistemas Distribuídos ...,A,MM,OPT,Informática ...,51799,2631,Doutorado,-2
17,20102,120149796,Leonardo Luiz Barbosa Vieira Cruciol ...,316547,Proficiência em Inglês ...,A,AP,OBR,Informática ...,51799,2631,Doutorado,-3
20,20102,120178893,Ana Cristina Rocha Oliveira Valverde ...,316296,Teoria da Computação ...,A,MS,OBR,Informática ...,51799,2631,Doutorado,-3
21,20102,130050652,Emerson de Araújo Macedo ...,316504,Tópicos em Fundamentos e Métodos de Computação...,A,MS,OPT,Informática ...,51799,2631,Doutorado,-4
22,20102,130051241,Daniel Sundfeld Lima ...,316296,Teoria da Computação ...,A,MM,OBR,Informática ...,51799,2631,Doutorado,-4


### Matrículas em todas as disciplinas do PPGI

In [479]:
disciplinas = df.groupby(['Disciplina', 'Código da Disciplina']).size()

fig = px.pie(df, names='Disciplina', title='Matrículas em todas as disciplinas do PPGI')
fig.show()

In [480]:
fig = px.pie(df, names='Menção', title='Menções obtidas pelos alunos em todas as disciplinas cursadas')
fig.show()

In [481]:
fig = px.pie(df[df.Menção == 'CC'], names='Disciplina', title='Menções Crédito Concedido nas disciplinas do PPGI')
fig.show()

## Análise das matrículas resultantes em reprovações e desistências

In [482]:
mi = df['Menção'] == "MI"

ii = df['Menção'] == "II"

sr = df['Menção'] == "SR"

dfrep = df[mi | ii | sr ]

fig = px.pie(dfrep, names='Disciplina', title='Números absolutos das reprovações e desistências nas disciplinas do PPGI')
fig.show()

In [483]:
fig = px.pie(df[df.Menção == 'MM'], names='Disciplina', title='Matrículas em disciplinas que obtiveram menção MM')
fig.show()

## Menções dos exames de qualificação

In [484]:
fig = px.pie(df[df['Código da Disciplina'] == 316580], names='Menção', title='Menções em Exame de Qualificação')
fig.show()

## Gerar um gráfico do tempo que leva para concluir o exame de qualificação

In [485]:
quali = df[df['Código da Disciplina'] == 316580]

#ferro = quali[quali['Matrícula'] == 180061712]

#ferro
tempoQualificar = quali[['Matrícula','Tempo']]

#print()
tempoQualificar['id'] = np.arange(len(tempoQualificar)) + 1

fig = px.bar(tempoQualificar, x="Tempo", y="id", orientation='h')
fig.show()



/home/vinicius/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Disciplinas cursadas com maior quantidade de matrículas

In [486]:
propdf = df;

propdf = propdf.drop(['Código da Disciplina'], axis=1)

propdf = propdf[propdf.Menção != 'AP']

propdf = propdf[propdf.Menção != 'CC']

propdf = propdf[propdf.Menção != 'DP']

propdf = propdf[propdf.Menção != 'RP']

totalDisc = propdf.groupby(['Disciplina']).size()

#print(totalDisc)

## Resposta: Q2

In [487]:
prop = propdf.groupby(['Disciplina','Menção']).size() / propdf.groupby(['Disciplina']).size()

print(prop)

Disciplina                                                              Menção
Arquiteturas VLSI                                                       MS        0.800000
                                                                        SS        0.200000
Engenharia de Software 1                                                SS        1.000000
Engenharia de Software 2                                                MS        0.125000
                                                                        SS        0.875000
Estágio de Docência no Ensino de Graduação em Ciência da Computação     MM        0.012195
                                                                        MS        0.012195
                                                                        SS        0.963415
                                                                        TR        0.012195
Fundamentos de Sistemas Computacionais                                  MI        0.060241
           

### Drop course's name attribute (we refer them by their own IDs)

In [488]:
precdf = df

precdf = precdf[precdf.Menção != 'AP']

precdf = precdf[precdf.Menção != 'CC']

precdf = precdf[precdf.Menção != 'DP']

precdf = precdf[precdf.Menção != 'RP']

In [489]:
mencoes = precdf.groupby(['Menção']).size()

fig = px.pie(precdf, names='Menção', title='Contabilização e Proporção das Menções em relação ao total de matrículas em disciplinas')
fig.show()

In [490]:
precdf = precdf.rename(columns={'Período':'periodo', 'Código da Disciplina': 'codigo_disciplina','Menção' : 'mencao'})

precdf['mencao'] = precdf['mencao'].astype('category')

cat_columns = precdf.select_dtypes(['category']).columns

precdf[cat_columns] = precdf[cat_columns].apply(lambda x: x.cat.codes)

#print(precdf)

#print(cat_columns)

### Dummy variables

In [491]:
#pd.get_dummies(df, prefix=['Código da Disciplina'])
mencoes = precdf.groupby(['mencao']).size()

print(mencoes)

mencoes = df.groupby(['Menção']).size()

print(mencoes)

mencao
0      5
1     11
2    114
3    258
4      6
5    385
6      8
dtype: int64
Menção
AP    120
CC     99
DP      9
II      5
MI     11
MM    114
MS    258
RP      1
SR      6
SS    385
TR      8
dtype: int64


### Visualization based on Parallel Coordinates

In [492]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = precdf['mencao'],
                     colorscale=["orange","red","pink","green","purple","blue","black"]),
        dimensions = list([
            dict(range = [20102,20192],
                label = 'Período', values = precdf['periodo']),
            dict(range = [316000,316700],
                label = 'Código da Disciplina', values = precdf['codigo_disciplina']),
            dict(range = [0,9],
                label = 'Tempo', values = precdf['Tempo'])
        ])
    )
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white'
)
fig.show()


In [493]:
#coursesArray = pd.unique(df['Disciplina'].values)
#courses = coursesArray.tolist()

#discNames = {}

#invdict = {}
#for i in range(0,len(courses)):
#    discNames[i] = courses[i]
#    invdict[courses[i]] = i

#print(discNomes)

#codesArray = pd.unique(df['Código da Disciplina'].values)
#codes = codesArray.tolist()

#discCodes = {}

#invdict2 = {}
#for i in range(0,len(codes)):
#    discCodes[i] = codes[i]
#    invdict2[codes[i]] = i


#for i in range(0,len(courses)):
#    print(str(discCodes[i])+' '+str(discNames[i]))
